In [ ]:
from icp_warm_up.utils import icp, o3d_icp
from pr2_utils import *

In [ ]:
dataset = 20
with np.load(f"../data/Hokuyo{dataset}.npz") as data:
    lidar_ranges = data["ranges"].T       # range data [m] (Note: values < range_min or > range_max should be discarded)
    lidar_stamps = data["time_stamps"]  # acquisition times of the lidar scans

with np.load(f"../data/odometry_imu_{dataset}.npz") as data:
    odometry = data["X"]
    odometry_stamp = data["stamps"]

f_odometry = InterpN1D(odometry_stamp, odometry)

In [ ]:
plot_scan(lidar_ranges[1400], f_odometry(lidar_stamps[1400]))
plot_scan(lidar_ranges[1401], f_odometry(lidar_stamps[1401]))

o3d_icp(
    lidar_scan_to_3dpc(lidar_ranges[1401,:]),
    lidar_scan_to_3dpc(lidar_ranges[1400,:]),
)

In [ ]:
n = lidar_stamps.shape[0]
T_scan = np.zeros([n,4,4])
T_scan[0] = np.eye(4)
T_odo = odometry_to_transformation(odometry)
sum_error = 0
for i in tqdm(range(1, n)):
    T_guess = diff_transformation(T_odo[i], T_odo[i-1])
    T, error = icp(
        lidar_scan_to_3dpc(lidar_ranges[i,:]),
        lidar_scan_to_3dpc(lidar_ranges[i-1,:]),
        T_guess
    )
    sum_error += error
    T_scan[i] = T_scan[i-1] @ T
print(sum_error / n)
odometry_scan = transformation_to_odometry(T_scan)

In [ ]:
with np.load(f"../data/odometry_icp_{dataset}.npz") as data:
    odometry_scan = data["X"]

In [ ]:
plot_odometry([
    (odometry,odometry_stamp, "imu (motion)"),
    (odometry_scan, lidar_stamps, "icp (observation)")
])

In [ ]:
np.savez(f'../data/odometry_icp_{dataset}.npz', X=odometry_scan, stamps=lidar_stamps)